# Step 5: Add a deployment pipeline

In previous step you implemented an automated data processing and model building pipeline. Each run of the pipeline produces a new version of the model. This notebook implements the automated model deployment step in our ML workflow.

You use the second part of the [SageMaker MLOps project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-git-code-pipeline) to create a ready-to use model deployment CI/CD pipeline.

The MLOps project template implements a CodePipeline pipeline for deployment of models in the SageMaker model registry to SageMaker endpoints for real-time inference. When a new model version is registered and approved in the model registry, the pipeline automatically initiates and performs a deployment.

||||
|---|---|---|
|1. |Experiment in a notebook ||
|2. |Scale with SageMaker AI processing jobs and SageMaker SDK ||
|3. |Operationalize with ML pipeline, model registry, and feature store ||
|4. |Add a model building CI/CD pipeline ||
|5. |Add a model deployment pipeline |**<<<< YOU ARE HERE**|
|6. |Add model and data monitoring ||

# Step 5: 배포 파이프라인 추가

이전 단계에서 자동화된 데이터 처리 및 모델 구축 파이프라인을 구현했습니다. 파이프라인의 각 실행은 새로운 버전의 모델을 생성합니다. 이 노트북은 ML 워크플로우에서 자동화된 모델 배포 단계를 구현합니다.

[SageMaker MLOps 프로젝트 템플릿](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-sm.html#sagemaker-projects-templates-git-code-pipeline)의 두 번째 부분을 사용하여 바로 사용할 수 있는 모델 배포 CI/CD 파이프라인을 생성합니다.

MLOps 프로젝트 템플릿은 SageMaker 모델 레지스트리의 모델을 실시간 추론을 위한 SageMaker 엔드포인트에 배포하기 위한 CodePipeline 파이프라인을 구현합니다. 새로운 모델 버전이 모델 레지스트리에 등록되고 승인되면, 파이프라인이 자동으로 시작되어 배포를 수행합니다.

||||
|---|---|---|
|1. |노트북에서 실험 ||
|2. |SageMaker AI 처리 작업과 SageMaker SDK로 확장 ||
|3. |ML 파이프라인, 모델 레지스트리, 피처 스토어로 운영화 ||
|4. |모델 구축 CI/CD 파이프라인 추가 ||
|5. |모델 배포 파이프라인 추가 |**<<<< 현재 위치**|
|6. |모델 및 데이터 모니터링 추가 ||

이 단계에서는:
- 자동화된 모델 배포 구현
- 실시간 추론 엔드포인트 설정
- 모델 버전 관리 자동화
- 배포 프로세스 자동화

<div class="alert alert-info"> Make sure you using <code>Python 3</code> kernel in JupyterLab for this notebook.</div>

In [1]:
import boto3
import sagemaker 
from time import gmtime, strftime, sleep
from IPython.display import HTML

sagemaker.__version__

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Fetched defaults config from location: /home/sagemaker-user/.config/sagemaker/config.yaml


'2.239.0'

In [2]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
baseline_s3_url                         -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
bucket_name                             -> 'sagemaker-us-east-1-906545278380'
bucket_prefix                           -> 'from-idea-to-prod/xgboost'
dataset_feature_group_name              -> 'from-idea-to-prod-12-06-30-53'
dataset_file_local_path                 -> 'data/bank-additional/bank-additional-full.csv'
domain_id                               -> 'd-igloxuzrs3z2'
evaluation_s3_url                       -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
experiment_name                         -> 'from-idea-to-prod-experiment-11-21-33-07'
feature_store_bucket_prefix             -> 'from-idea-to-prod/feature-store'
initialized                             -> True
input_s3_url                            -> 's3://sagemaker-us-east-1-906545278380/from-idea-t
mlflow_arn                              -> 'arn:aws:sagemaker:us-east-1:906545278380:mlflow

In [3]:
sm = boto3.client("sagemaker")

In [48]:
try:
    print(project_name)
    print(project_id)
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("You must complete the notebook 04-sagemaker-project")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++")

mlops-02-14-11-03-33
p-ca7phmcraepa


## Explore the project in the Studio UI

<div class="alert alert-info">Make sure you run the step 4 notebook and successfully provisioned the MLOps project and run the project pipeline at least once.</div>

Click on the links constructed by the next code cell to see the project and the model package in the Studio UI. You must see at least one version of the model registered in the model package group.

## Studio UI에서 프로젝트 탐색

<div class="alert alert-info">4단계 노트북을 실행하고 MLOps 프로젝트를 성공적으로 프로비저닝하고 프로젝트 파이프라인을 최소 한 번 실행했는지 확인하세요.</div>

다음 코드 셀에서 생성된 링크를 클릭하여 Studio UI에서 프로젝트와 모델 패키지를 확인하세요. 모델 패키지 그룹에 최소 하나의 모델 버전이 등록되어 있어야 합니다.

주요 확인 사항:
1. 사전 조건 확인
   - 4단계 노트북 실행 완료
   - MLOps 프로젝트 설정 완료
   - 파이프라인 최소 1회 실행

2. UI 확인 항목
   - 프로젝트 상태
   - 모델 패키지 정보
   - 등록된 모델 버전

3. 진행 상태 검증
   - 모델 등록 확인
   - 버전 정보 확인
   - 설정 상태 점검

4. 다음 단계 준비
   - 모델 배포 준비
   - 파이프라인 구성 확인

In [5]:
# Show the project link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/projects/{}/">the project</a> in the Studio UI</b>'.format(
            domain_id, region, project_name))
)

# Show the model package link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}-{}/versions">the model package versions</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_id))
)

In [20]:
# check that the project exists
project_data = sm.describe_project(ProjectName=project_name)

In [21]:
assert project_data['ProjectStatus'] == 'CreateCompleted', 'Project must be created at this point!'

In [24]:
# check that at least one model version is registered in the model registry
model_packages = sm.list_model_packages(
    ModelPackageGroupName=f'{project_name}-{project_id}',
    ModelApprovalStatus='PendingManualApproval')

In [25]:
assert len(model_packages['ModelPackageSummaryList']) > 0, 'You must have at least one model version in the status PendingManualApproval'

---

## Working with MLOps project for model deployment
The template provisions a GitHub repository with configuration files to specify the model deployment steps, AWS CloudFormation templates to define endpoints as infrastructure, and seed code for testing the endpoint.

This template provides the following resources:

1. A GitHub repository that contains seed code that deploys models to endpoints in staging and production environments
2. An AWS CodePipeline pipeline that has `source`, `build`, `deploy-to-staging`, and `deploy-to-production` steps. The `source` step points to the GitHub repository, and the `build` step gets the code from that repository and generates CloudFormation stacks to deploy. The `deploy-to-staging` and `deploy-to-production` steps deploy the CloudFormation stacks to their respective environments. There is a manual approval step between the staging and production build steps, so that a MLOps engineer must approve the model before it is deployed to production.
3. An Amazon EventBridge rule to launch a CodePipeline pipeline execution when a model package version is approved or rejected.
4. There is also a manual approval step after the placeholder unit tests. You can implement your own tests to replace the placeholders tests.

The project template also deploys an Amazon S3 bucket to store artifacts, including CodePipeline and CodeBuild artifacts, and any artifacts generated from the SageMaker pipeline runs.

The following diagram shows the architecture.

<img src="img/mlops-model-deploy.png" width="600"/>

You don't need to implement any configuration changes for the project. The model deployment pipeline works out of the box.
To start the model deployment pipeline, you must approve the model version in the model registry.

## 모델 배포를 위한 MLOps 프로젝트 작업
템플릿은 모델 배포 단계를 지정하는 구성 파일, 인프라로서 엔드포인트를 정의하는 AWS CloudFormation 템플릿, 엔드포인트 테스트를 위한 시드 코드가 포함된 GitHub 저장소를 프로비저닝합니다.

이 템플릿은 다음 리소스를 제공합니다:

1. 스테이징 및 프로덕션 환경에 모델을 배포하는 시드 코드가 포함된 GitHub 저장소
2. `source`, `build`, `deploy-to-staging`, `deploy-to-production` 단계가 있는 AWS CodePipeline 파이프라인
   - `source` 단계는 GitHub 저장소를 가리킴
   - `build` 단계는 해당 저장소에서 코드를 가져와 배포용 CloudFormation 스택 생성
   - `deploy-to-staging`과 `deploy-to-production` 단계는 각 환경에 CloudFormation 스택 배포
   - 스테이징과 프로덕션 빌드 단계 사이에 수동 승인 단계가 있어 MLOps 엔지니어가 프로덕션 배포 전 모델을 승인해야 함
3. 모델 패키지 버전이 승인되거나 거부될 때 CodePipeline 파이프라인 실행을 시작하는 Amazon EventBridge 규칙
4. 플레이스홀더 단위 테스트 후 수동 승인 단계도 있음. 플레이스홀더 테스트를 자체 테스트로 대체 가능

프로젝트 템플릿은 또한 다음을 저장하기 위한 Amazon S3 버킷을 배포합니다:
- CodePipeline 및 CodeBuild 아티팩트
- SageMaker 파이프라인 실행에서 생성된 아티팩트

다음 다이어그램은 아키텍처를 보여줍니다:

<img src="img/mlops-model-deploy.png" width="600"/>

프로젝트에 대한 구성 변경을 구현할 필요가 없습니다. 모델 배포 파이프라인은 즉시 사용 가능합니다.
모델 배포 파이프라인을 시작하려면 모델 레지스트리에서 모델 버전을 승인해야 합니다.

### Approve a model version
Approving a model version causes the MLOps project to launch the model deployment process. 

In the first step, the model deployment pipeline deploys the model version to a staging SageMaker real-time inference end-point.

You can approve the model version either in Studio in the Model registry or do it programmatically in the notebook. Let's do it programatically.

### 모델 버전 승인
모델 버전을 승인하면 MLOps 프로젝트가 모델 배포 프로세스를 시작합니다.

첫 번째 단계에서, 모델 배포 파이프라인은 모델 버전을 스테이징 SageMaker 실시간 추론 엔드포인트에 배포합니다.

모델 버전은 Studio의 Model registry에서 승인하거나 노트북에서 프로그래밍 방식으로 승인할 수 있습니다. 프로그래밍 방식으로 진행해 보겠습니다.

주요 내용:
1. 승인 프로세스
   - 자동 배포 트리거
   - 모델 버전 관리

2. 배포 단계
   - 스테이징 환경 우선 배포
   - 실시간 추론 엔드포인트 설정

3. 승인 방법
   - Studio UI 사용
   - 프로그래밍 방식 사용

4. 자동화된 워크플로우
   - 승인 후 자동 배포
   - 단계별 검증 진행

In [32]:
# list all model packages and select the latest one
model_packages = [
    p["ModelPackageSummaryList"] for p in
    sm.get_paginator('list_model_packages').paginate(
        ModelPackageGroupName=f'{project_name}-{project_id}',
        ModelApprovalStatus='PendingManualApproval',
        SortBy="CreationTime",
        SortOrder="Descending",
    )
][0]

In [33]:
if len(model_packages) == 0:
    raise Exception(f"No model package is found for {model_package_group_name} model package group. Run a model creation pipeline first.")

print(f"There are {len(model_packages)} model versions in the {model_package_group_name} model package group")
print(f"Approve the most recent model package:")

latest_model_package_arn = model_packages[0]["ModelPackageArn"]
print(latest_model_package_arn)

There are 2 model versions in the from-idea-to-prod-pipeline-model-12-06-30-22 model package group
Approve the most recent model package:
arn:aws:sagemaker:us-east-1:906545278380:model-package/mlops-02-14-11-03-33-p-ca7phmcraepa/2


The following statement sets the `ModelApprovalStatus` for the most recent model package in the model registry to `Approved`. The model package state change launches the EventBridge rule and the rule launches the CodePipeline CI/CD pipeline with model deployment.

In [34]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=latest_model_package_arn,
    ModelApprovalStatus="Approved",
)

Click on the link constructed by the following code cell to see the last model version in the model registry in the Studio UI changed the **Status** to `Approved`:

In [40]:
# Show the model package version link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/models/registered-models/{}-{}/versions/version-{}/overview">the model package version</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_id, latest_model_package_arn.split('/')[-1]))
)

### Deployment pipeline execution
Upon approval of a model version in the code cell above, the model deployment CI/CD pipeline performs the following actions:
1. Create CloudFormation parameter configuration files with the staging and production parameters for CloudFormation templates for SageMaker endpoint
1. Create a SageMaker real-time inference endpoint with the name `<PROJECT-NAME>-staging`
1. Run the test script on the staging endpoint
1. Wait until the test result is manually approved in [AWS CodePipeline console](https://console.aws.amazon.com/codesuite/codepipeline)
1. Create a SageMaker endpoint with the name `<PROJECT-NAME>-prod` in the current account

### 배포 파이프라인 실행
위 코드 셀에서 모델 버전이 승인되면, 모델 배포 CI/CD 파이프라인은 다음 작업을 수행합니다:
1. SageMaker 엔드포인트용 CloudFormation 템플릿의 스테이징 및 프로덕션 매개변수로 CloudFormation 매개변수 구성 파일 생성
1. `<PROJECT-NAME>-staging` 이름으로 SageMaker 실시간 추론 엔드포인트 생성
1. 스테이징 엔드포인트에서 테스트 스크립트 실행
1. [AWS CodePipeline 콘솔](https://console.aws.amazon.com/codesuite/codepipeline)에서 테스트 결과가 수동으로 승인될 때까지 대기
1. 현재 계정에 `<PROJECT-NAME>-prod` 이름으로 SageMaker 엔드포인트 생성

주요 단계:
1. 구성 파일 준비
   - CloudFormation 매개변수 설정
   - 환경별 구성 관리

2. 스테이징 환경 설정
   - 실시간 추론 엔드포인트 생성
   - 초기 테스트 환경 구성

3. 검증 프로세스
   - 테스트 스크립트 실행
   - 결과 검증 및 승인

4. 프로덕션 배포
   - 승인 후 프로덕션 엔드포인트 생성
   - 실제 서비스 환경 구성

Wait about 5-10 minutes until the pipeline finishes deployment of the staging endpoint. You can see the status of the endpoint in the Studio UI in **Deployments** > **Endpoints**:

파이프라인이 스테이징 엔드포인트 배포를 완료할 때까지 약 5-10분 기다리세요. Studio UI의 **Deployments** > **Endpoints**에서 엔드포인트의 상태를 확인할 수 있습니다:

주요 모니터링 포인트:
1. 배포 진행 상황
   - Studio UI에서 실시간 확인
   - 엔드포인트 상태 모니터링

2. 확인 위치
   - Deployments 메뉴
   - Endpoints 섹션
   - 상태 표시기

3. 소요 시간
   - 약 5-10분 예상
   - 완료될 때까지 대기 필요

4. 상태 확인
   - 엔드포인트 생성 상태
   - 서비스 가용성
   - 배포 완료 여부

In [41]:
# Show the inference endpoints link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">inference endpoints</a> in the Studio UI</b>'.format(
            domain_id, region))
)

After the endpoint status changed from `Creating` to `InService`, the staging endpoint is fully operational. You can launch the model deployment process to the production stage by manually approving the **DeployStaging** stage of the CodePipeline pipeline. In the next section you approve the model deployment and launch the second stage of the deployment into a production endpoint.

엔드포인트 상태가 `Creating`에서 `InService`로 변경된 후, 스테이징 엔드포인트가 완전히 작동 가능한 상태가 됩니다. CodePipeline 파이프라인의 **DeployStaging** 단계를 수동으로 승인하여 프로덕션 단계로의 모델 배포 프로세스를 시작할 수 있습니다. 다음 섹션에서는 모델 배포를 승인하고 프로덕션 엔드포인트로의 두 번째 배포 단계를 시작합니다.

주요 단계:
1. 스테이징 배포 완료 확인
   - 상태 변경 확인 (Creating → InService)
   - 엔드포인트 작동 상태 검증

2. 프로덕션 배포 준비
   - DeployStaging 단계 승인 필요
   - 수동 승인 프로세스

3. 다음 단계
   - 프로덕션 배포 시작
   - 두 번째 배포 단계 실행

4. 전환 과정
   - 스테이징에서 프로덕션으로
   - 단계적 배포 관리

### Deploy the model version to production

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">Wait until staging endpoint status changes to InService, then continue with the following code cells.
    </p>
</div>

### 모델 버전을 프로덕션에 배포

<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">스테이징 엔드포인트 상태가 InService로 변경될 때까지 기다린 후, 다음 코드 셀을 계속 진행하세요.
    </p>
</div>

주요 안내:
1. 배포 전 필수 확인사항
   - 스테이징 엔드포인트 상태 확인
   - InService 상태 대기

2. 진행 순서
   - 상태 확인
   - 코드 셀 실행
   - 프로덕션 배포 시작

3. 주의사항
   - 상태 확인 필수
   - 순서 준수 중요
   - 적절한 대기 시간 필요

4. 다음 단계 준비
   - 프로덕션 배포 준비
   - 코드 실행 대기

In [49]:
while sm.describe_endpoint(EndpointName=f'{project_name}-staging')['EndpointStatus'] != 'InService':
    sleep(10)
    print(f'Waiting untile the endpoint {project_name}-staging is operational')

print(f'Endpoint {project_name}-staging is InService')

Endpoint mlops-02-14-11-03-33-staging is InService


The next code cell construct a CodePipeline approval link - click on the link to to open the CodePipeline console with the pipeline execution workflow.

다음 코드 셀은 CodePipeline 승인 링크를 생성합니다 - 링크를 클릭하여 파이프라인 실행 워크플로우가 있는 CodePipeline 콘솔을 엽니다.

주요 내용:
1. 링크 생성
   - CodePipeline 승인 링크 생성
   - 콘솔 직접 접근 가능

2. 워크플로우 확인
   - 파이프라인 실행 상태 확인
   - 단계별 진행 상황 모니터링

3. 승인 절차
   - 콘솔에서 직접 승인 가능
   - 워크플로우 제어 가능

4. 접근 방법
   - 생성된 링크 클릭
   - 파이프라인 콘솔로 이동
   - 승인 프로세스 진행

In the **DeployStaging stage**, choose **Review** on the **ApproveDeployment** step. You might wait until `TestStaging` step completes with `Succeeded` status. 

<img src="img/deploy-staging-review.png" alt="approve deployment" width="800px">

In the **Review** dialog box, select **Approve** and choose **Submit**.

**DeployStaging 단계**에서 **ApproveDeployment** 단계의 **Review**를 선택하세요. `TestStaging` 단계가 `Succeeded` 상태로 완료될 때까지 기다려야 할 수 있습니다.

<img src="img/deploy-staging-review.png" alt="배포 승인" width="800px">

**Review** 대화 상자에서 **Approve**를 선택하고 **Submit**을 클릭하세요.

주요 단계:
1. 승인 절차 시작
   - DeployStaging 단계 확인
   - ApproveDeployment 검토

2. 상태 확인
   - TestStaging 완료 대기
   - Succeeded 상태 확인

3. 승인 실행
   - Review 대화 상자 열기
   - Approve 선택
   - Submit으로 확정

4. 다음 단계
   - 승인 후 자동 진행
   - 프로덕션 배포 시작

In [57]:
# Show the approval link
display(
    HTML(
        '<b>Please approve the manual step in <a target="top" href="https://console.aws.amazon.com/codesuite/codepipeline/pipelines/sagemaker-{}-{}-modeldeploy/view?region={}">AWS CodePipeline</a></b>'.format(
            project_name, project_id, region)
    )
)

Approving the **DeployStaging** stage causes the deployment pipeline to continue and to deploy the model to the production endpoint. To view the endpoints, choose the **Deployments** > **Endpoints** in Studio UI or click on the link below.

**DeployStaging** 단계를 승인하면 배포 파이프라인이 계속 진행되어 모델을 프로덕션 엔드포인트에 배포합니다. 엔드포인트를 보려면 Studio UI에서 **Deployments** > **Endpoints**를 선택하거나 아래 링크를 클릭하세요.

주요 내용:
1. 배포 진행
   - DeployStaging 승인 후 자동 진행
   - 프로덕션 엔드포인트 배포 시작

2. 엔드포인트 확인 방법
   - Studio UI 네비게이션
   - Deployments 메뉴
   - Endpoints 섹션 선택

3. 모니터링 가능 항목
   - 스테이징 엔드포인트 상태
   - 프로덕션 엔드포인트 생성
   - 배포 진행 상황

4. 접근 옵션
   - UI를 통한 확인
   - 제공된 링크 사용
   - 실시간 상태 모니터링

In [58]:
# Show the inference endpoints link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/inference-experience/endpoints">inference endpoints</a> in the Studio UI</b>'.format(
            domain_id, region))
)

As your CI/CD deployment pipeline continues, you see the production endpoint in status `Creating` along with the previously deployed staging endpoint in status `InService`.

After 5-10 min the deployment is completed and both endpoints are in status `InService`.

Click on the link constructed by the following cell to see the endpoints that belongs to the project.
You see that both endpoints, `staging` and `prod`, are visible in the deployment project because the project and the endpoints are connected via the metadata and resource tags.

CI/CD 배포 파이프라인이 계속 진행됨에 따라, 이전에 배포된 스테이징 엔드포인트가 `InService` 상태인 동시에 프로덕션 엔드포인트가 `Creating` 상태인 것을 확인할 수 있습니다.

5-10분 후 배포가 완료되면 두 엔드포인트 모두 `InService` 상태가 됩니다.

다음 셀에서 생성된 링크를 클릭하여 프로젝트에 속한 엔드포인트들을 확인하세요.
`staging`과 `prod` 두 엔드포인트 모두 메타데이터와 리소스 태그를 통해 프로젝트와 연결되어 있기 때문에 배포 프로젝트에서 확인할 수 있습니다.

주요 내용:
1. 배포 상태
   - 스테이징: InService
   - 프로덕션: Creating → InService

2. 소요 시간
   - 완료까지 5-10분 소요
   - 두 엔드포인트 모두 활성화

3. 프로젝트 연계
   - 메타데이터 연결
   - 리소스 태그 활용
   - 통합된 뷰 제공

4. 확인 방법
   - 생성된 링크로 접근
   - 두 환경 모두 확인
   - 상태 모니터링 가능

In [59]:
# Show the project link
display(
    HTML('<b>See <a target="top" href="https://studio-{}.studio.{}.sagemaker.aws/projects/{}/endpoints">endpoints of the project {}</a> in the Studio UI</b>'.format(
            domain_id, region, project_name, project_name))
)

## Summary
In this notebook you implemented an automated CI/CD deployment pipeline with the following features:
- use CloudFormation templates for SageMaker real-time inference endpoint deployment
- model approval in the model registry launches the model deployment pipeline
- model deployment pipeline contains two stages, staging and production with automated tests for the staging endpoint and manual approval for the production deployment, and final deployment of the production endpoint.

## 요약
이 노트북에서는 다음과 같은 기능을 갖춘 자동화된 CI/CD 배포 파이프라인을 구현했습니다:
- SageMaker 실시간 추론 엔드포인트 배포를 위한 CloudFormation 템플릿 사용
- 모델 레지스트리의 모델 승인이 모델 배포 파이프라인을 시작
- 모델 배포 파이프라인은 다음 두 단계를 포함:
  - 스테이징 단계: 자동화된 테스트 포함
  - 프로덕션 단계: 수동 승인 및 최종 프로덕션 엔드포인트 배포

주요 구현 사항:
1. 인프라 자동화
   - CloudFormation 템플릿 활용
   - 엔드포인트 자동 구성

2. 자동화된 워크플로우
   - 모델 승인 기반 트리거
   - 단계별 배포 자동화

3. 이중 환경 구성
   - 스테이징 환경 테스트
   - 프로덕션 환경 보호

4. 품질 관리
   - 자동화된 테스트
   - 수동 승인 프로세스
   - 안전한 프로덕션 배포

---

## Clean-up
<div style="border: 4px solid coral; text-align: center; margin: auto;">
    <p style=" text-align: center; margin: auto;">
    If you're going to run the step 6 notebook (Data and Model Quality Monitoring), you need to keep at least one of the endpoints. If you finish the workshop here and don't run the step 6 notebook, navigate to the <b>clean-up notebook (99-clean-up.ipynb)</b> and follow the clean-up instructions to avoid charges in your AWS account.
    <br>
    <br>
    You don't need to run the clean-up if you're using an AWS-provided AWS account.
    </p>
</div>

## Continue with the step 6
open the step 6 [notebook](06-monitoring.ipynb).

## Further development ideas for your real-world projects
- Add end-to-end data encryption using AWS KMS keys
- Create a [custom SageMaker project template](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-templates-custom.html) for model deployment to cover your specific project requirements
- Add [multi-account model deployment](https://aws.amazon.com/blogs/machine-learning/multi-account-model-deployment-with-amazon-sagemaker-pipelines/) to your ML workflow
- Add automated model tests to the placeholder in the CodePipeline pipeline
- Use [Amazon SageMaker Inference Recommender](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-recommender.html) to run automated load tests for your inference endpoints and to select the best instance type and configuration

## Additional resources
- [Deploy a Machine Learning Model to a Real-Time Inference Endpoint](https://aws.amazon.com/getting-started/hands-on/machine-learning-tutorial-deploy-model-to-real-time-inference-endpoint/)
- [SageMaker MLOps Project Walkthrough](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-projects-walkthrough.html)
- [Amazon SageMaker Pipelines lab in SageMaker Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/63069e26-921c-4ce1-9cc7-dd882ff62575/en-US/lab6)
- [Amazon SageMaker secure MLOps](https://github.com/aws-samples/amazon-sagemaker-secure-mlops)
- [Testing approaches for Amazon SageMaker ML models](https://aws.amazon.com/blogs/machine-learning/testing-approaches-for-amazon-sagemaker-ml-models/)
- [Model hosting patterns in Amazon SageMaker blog series](https://aws.amazon.com/blogs/machine-learning/model-hosting-patterns-in-amazon-sagemaker-part-1-common-design-patterns-for-building-ml-applications-on-amazon-sagemaker/)
- [Take advantage of advanced deployment strategies using Amazon SageMaker deployment guardrails](https://aws.amazon.com/blogs/machine-learning/take-advantage-of-advanced-deployment-strategies-using-amazon-sagemaker-deployment-guardrails/)
- [MLOps deployment best practices for real-time inference model serving endpoints with Amazon SageMaker](https://aws.amazon.com/blogs/machine-learning/mlops-deployment-best-practices-for-real-time-inference-model-serving-endpoints-with-amazon-sagemaker/)

# Shutdown kernel

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>